In [1]:
import numpy as np
import os
import nrrd
import h5py
import pandas
import json
from data import *

/home/davech2y/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# database_root = "/mnt/raid/davech2y/ShapeNetCore_vol/"
# database_name = "nrrd_256_filter_div_32_solid.hdf5"
# database = h5py.File(os.path.join(database_root, database_name), "w")

# shapenet_root = "/mnt/raid/davech2y/ShapeNetCore_vol/nrrd_256_filter_div_32_solid/"
# shapenet_csv = pandas.read_csv("captions.tablechair.csv")
# shapenet_shapepath = [
#     os.path.join(shapenet_root, model_name, model_name + '.nrrd') for model_name in shapenet_csv.modelId.values.tolist()
# ]
# shapenet_size = len(shapenet_shapepath)
# shapenet_shape = [3, 32, 32, 32]
# shapenet_shape_flat = np.prod(shapenet_shape)

# shapenet_dataset = database.create_dataset("shapes", (shapenet_size, shapenet_shape_flat), dtype="float")

# for i, shapepath in enumerate(shapenet_shapepath):
#     if i % 100 == 0: 
#         print("preprocessed and stored: ", i)
#     shape, _ = nrrd.read(shapepath)
#     shape = np.reshape(shape[:3], (-1))
#     shape = (shape - np.min(shape)) / (np.max(shape) - np.min(shape))
#     shapenet_dataset[i] = shape

In [2]:
coco_root = "/mnt/raid/davech2y/COCO_2014/"
coco_train = os.path.join(coco_root, "train2014")
coco_valid = os.path.join(coco_root, "valid2014")
coco_test = os.path.join(coco_root, "test2014")
coco_train_cap = os.path.join(coco_root, "annotations", "captions_train2014.json")
coco_valid_cap = os.path.join(coco_root, "annotations", "captions_val2014.json")

In [7]:
coco_valid = COCO(pandas.read_csv(os.path.join(coco_root, "preprocessed", "coco_val2014.caption.csv")), 100)

In [13]:
pandas.read_csv(os.path.join(coco_root, "preprocessed", "coco_val2014.caption.csv")).sample(frac=1).reset_index(drop=True)

,image_id,caption,file_name
0,338417,A crowd of people standing at a social get tog...,COCO_val2014_000000338417.jpg
1,206356,A baseball player is holding a bat in preparat...,COCO_val2014_000000206356.jpg
2,58344,there is a large building that has two clocks ...,COCO_val2014_000000058344.jpg
3,340884,a elephant walking in a dusty and rocky area,COCO_val2014_000000340884.jpg
4,293057,A person on a snowboard jumping off a pile of ...,COCO_val2014_000000293057.jpg
5,173772,Group of people playing soccer in the snow.,COCO_val2014_000000173772.jpg
6,363039,this is a bird flying in the sky with its wings,COCO_val2014_000000363039.jpg
7,231097,A plate is left sitting half eaten with mostly...,COCO_val2014_000000231097.jpg
8,388225,A man and a woman enjoying the Nintendo Wii.,COCO_val2014_000000388225.jpg
9,285645,a person riding skis holding on to a parachute,COCO_val2014_000000285645.jpg


In [9]:
coco_valid_ds = COCOCaptionDataset(
    coco_root, 
    coco_valid.transformed_csv,
    os.path.join(coco_root, "preprocessed", "coco_val2014.hdf5")
)

In [10]:
coco_valid_ds.data_pairs

[(71988,
  1,
  [2,
   1,
   135,
   9,
   1,
   21,
   13,
   7,
   136,
   9,
   1,
   80,
   16,
   137,
   1,
   138,
   58,
   81,
   139,
   5,
   82,
   3],
  22),
 (322141,
  3,
  [2, 44, 11, 1, 36, 7, 22, 15, 6, 1, 59, 14, 7, 1, 141, 9, 5, 59, 4, 3, 0, 0],
  20),
 (71988,
  2,
  [2, 1, 34, 8, 1, 80, 7, 83, 16, 9, 1, 21, 35, 11, 140, 5, 43, 4, 3, 0, 0, 0],
  19),
 (517069,
  4,
  [2, 1, 23, 16, 9, 1, 21, 7, 1, 23, 60, 84, 5, 21, 27, 1, 85, 86, 3, 0, 0, 0],
  19),
 (315702,
  0,
  [2,
   1,
   15,
   128,
   6,
   129,
   7,
   130,
   14,
   7,
   79,
   131,
   132,
   133,
   134,
   57,
   3,
   0,
   0,
   0,
   0,
   0],
  17),
 (517069,
  6,
  [2, 1, 23, 16, 9, 1, 21, 7, 1, 23, 60, 61, 149, 5, 85, 4, 3, 0, 0, 0, 0, 0],
  17),
 (301837,
  7,
  [2,
   1,
   88,
   20,
   11,
   46,
   8,
   150,
   10,
   22,
   13,
   151,
   13,
   7,
   24,
   4,
   3,
   0,
   0,
   0,
   0,
   0],
  17),
 (16977,
  5,
  [2,
   45,
   142,
   28,
   143,
   87,
   144,
   145,
   146,
 